In [5]:
from dotenv import load_dotenv,dotenv_values
import os

# Getting Envoronment Variables
env_name = ".env"
config = dotenv_values(env_name)


#   Retrieve Credentials from Azure Key Vault

In [6]:
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from openai import AzureOpenAI
from azure.search.documents import SearchClient

# Initializing Keyvault
key_vault = config['key_vault']
print(f"Key Vault Name: {key_vault}")
key_vault_url = f"https://{key_vault}.vault.azure.net"
credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url=key_vault_url, credential=credential)

# Embedding credentials
openai_key = secret_client.get_secret("Embedding-Key").value
openai_endpoint = secret_client.get_secret("Embedding-Endpoint").value

# Chat completion credentials
chat_key = secret_client.get_secret("OpenAI-API-Key").value
chat_endpoint = secret_client.get_secret("OpenAI-Endpoint").value

# Azure AI Search Credentials
database = secret_client.get_secret("Database").value
username = secret_client.get_secret("Username").value
password = secret_client.get_secret("Password").value
server = secret_client.get_secret("Server").value

# Azure AI Search Credentials
search_service_endpoint = secret_client.get_secret("Search-Service-Endpoint").value
cog_search_key = secret_client.get_secret("Search-Service-Api-Key").value
cog_search_name = secret_client.get_secret("Search-Service-Name").value
cog_search_index = secret_client.get_secret("Search-Service-Index").value
# search_client = SearchClient(endpoint=search_service_endpoint, index_name="search-review", credential=AzureKeyCredential(search_service_key))

# # Create an instance of the AzureOpenAI class
embedding_client = AzureOpenAI(azure_endpoint=openai_endpoint,api_key=openai_key,api_version="2024-02-15-preview")
chat_client = AzureOpenAI(azure_endpoint=chat_endpoint,api_key=chat_key,api_version="2024-02-15-preview")

# # Models

Key Vault Name: pharmacy-keyvault


## The samples follow a RAG pattern that include the following steps:

1. Add sample data to an Azure SQL database product
2. Create embeddings from the sample data using an Azure OpenAI Embeddings model
3. Link the Azure database product to Azure AI Search (for databases without native vector indexing)
3. Create a vector index on the embeddings
4. Perform vector similarity search
5. Perform question answering over the sample data using an Azure OpenAI Chat Completions model

This dataset consists of reviews of fine foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories.


In [7]:
import pyodbc


driver = '{ODBC Driver 18 for SQL Server}'

# Connection String
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Lets establish a connection
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

print("Connected to Azure SQL Database successfully!")


Connected to Azure SQL Database successfully!


In [3]:
# Loading Database
cursor.execute("DROP TABLE IF EXISTS food_review")

# Create Table
cursor.execute("CREATE TABLE food_review (Id integer, ProductId text, UserId text, ProfileName text, HelpfulnessDenominator integer, HelpfulnessNumerator integer, Score integer, Time integer, Summary text, Text text)")
print("Table created successfully........")

# create index
cursor.execute("CREATE INDEX idx_Id ON food_review(Id);")
print("Index created successfully........")


Table created successfully........
Index created successfully........


In [4]:
# Load Data
import numpy as np
import pandas as pd
df = pd.read_csv('Reviews_small.csv')

# Specify Batch size
batch_size = 40
table_name = "food_review"

# Split the dataframe into batches
batches = [df[i:i + batch_size] for i in range(0, len(df), batch_size)]

# Iterate over each batch and insert the data into the database
for batch in batches:
    #Convert the batch  dataframe to a list of tuples for bulk insertion
    rows = [tuple(row) for row in batch.itertuples(index=False)]

    # Define Query
    query = f"INSERT INTO {table_name} (Id, ProductId, UserId, ProfileName, HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary, Text) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.executemany(query, rows)

    

In [5]:
try:
    cursor.execute("SELECT count(Id) FROM food_review")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error Executing Query: {e}")

(99,)


In [6]:
# Retrieve data from the database

try:
    cursor.execute("SELECT id, CONCAT('productid: ', productid, ' ', 'score: ', score, ' ', 'text: ', text) AS concat FROM food_review;")
    rows = cursor.fetchall()
except Exception as e:
    print(f"Error executing query: {e}")
    

In [7]:
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from openai import AzureOpenAI
from azure.search.documents import SearchClient

# Initializing Keyvault
key_vault = config['key_vault']
print(f"Key Vault Name: {key_vault}")
key_vault_url = f"https://{key_vault}.vault.azure.net"
credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url=key_vault_url, credential=credential)

# Embedding credentials
openai_key = secret_client.get_secret("Embedding-Key").value
openai_endpoint = secret_client.get_secret("Embedding-Endpoint").value

# Chat completion credentials
chat_key = secret_client.get_secret("OpenAI-API-Key").value
chat_endpoint = secret_client.get_secret("OpenAI-Endpoint").value


# Azure AI Search Credentials
search_service_endpoint = secret_client.get_secret("Search-Service-Endpoint").value
cog_search_key = secret_client.get_secret("Search-Service-Api-Key").value
cog_search_name = secret_client.get_secret("Search-Service-Name").value
cog_search_index = secret_client.get_secret("Search-Service-Index").value
# search_client = SearchClient(endpoint=search_service_endpoint, index_name="search-review", credential=AzureKeyCredential(search_service_key))

# # Create an instance of the AzureOpenAI class
embedding_client = AzureOpenAI(azure_endpoint=openai_endpoint,api_key=openai_key,api_version="2024-02-15-preview")
chat_client = AzureOpenAI(azure_endpoint=chat_endpoint,api_key=chat_key,api_version="2024-02-15-preview")

# # Models

Key Vault Name: pharmacy-keyvault


In [8]:
import time
import pandas as pd

def createEmbeddings(text):
    try:
        response = embedding_client.embeddings.create(input=text, model='text-embedding-ada-002')
        embeddings = response.data[0].embedding  # Correctly access the embedding
        return embeddings
    except Exception as e:
        print(f"Error creating embeddings for text: {text}\nError: {e}")
        return None

content_embeddings = []
idx = []
sleep_timer = 1

for row in rows:
    print(f"Processing row: {row}")  # Here we try and debug the statement
    if not isinstance(row[1], str):
        row[1] = str(row[1])  # Convert to string if not already

    idx.append(row[0])
    embedding = createEmbeddings(row[1])
    if embedding:
        content_embeddings.append(embedding)

    if sleep_timer % 20 == 0:
        print("Waiting...")
        time.sleep(10)
    sleep_timer += 1

df = pd.DataFrame({'embeddings': content_embeddings}, index=idx)  # Storing embeddings in a dataframe
print(df)


Processing row: (1, 'productid: B001E4KFG0 score: 5 text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.')
Processing row: (2, 'productid: B00813GRG4 score: 1 text: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".')
Processing row: (3, 'productid: B000LQOCH0 score: 4 text: This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S.

## Store the embeddings in Azure Cognitive Search Vector Store
AzureCogSearch provides a simple interface to create a vector database, store and retrieve data using vector search. You can read more about here more about Vector Search.

There are two steps to store data in AzureCogSearch vector database:

First, we create the index (or schema) of the vector database
Second, we add the chunked documents and their embeddings to the vector datastore

In [9]:
import requests
import json

EMBEDDING_LENGTH = 1536

# Create Index for the Search with fields as id, and contentVector
url = f"https://{cog_search_name}.search.windows.net/indexes/{cog_search_index}?api-version=2023-10-01-Preview"
payload = json.dumps({
  "name": cog_search_index,
  "fields": [
    {
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "filterable": True
    },
    {
      "name": "contentVector",
      "type": "Collection(Edm.Single)",
      "searchable": True,
      "retrievable": True,
      "dimensions": EMBEDDING_LENGTH,
      "vectorSearchProfile": "my-vector-search-profile"
    }
  ],
  "vectorSearch": {
    "algorithms": [
      {
        "name": "my-hnsw-config",
        "kind": "hnsw",
        # "hnswParameters": {
        #   "m": 4,
        #   "efConstruction": 400,
        #   "metric": "cosine"
        # }
      }
    ],
    "profiles": [
       {
         "name": "my-vector-search-profile",
         "algorithm": "my-hnsw-config"
       }
     ]
  },
  "semantic": {
    "configurations": [
      {
        "name": "my-semantic-config",
        "prioritizedFields": {
          "prioritizedContentFields": [
            {
              "fieldName": "id"
            }
          ],
        }
      }
    ]
  }
})
headers = {
  'Content-Type': 'application/json',
  'api-key': cog_search_key
}

response = requests.request("PUT", url, headers=headers, data=payload)
print(response.status_code)

204


In [10]:
def batch_append_payload(df):
    # Append the data/payload of the embeddings in batches into Azure AI Seaech
    list_values = []
    for index, row in df.iterrows():
        list_values.append(
            {
            "id": str(index),
            "contentVector": row['embeddings'],
            "@search.action": "upload"
            }
        )
    print('payload of size {}'.format(len(list_values)))
    print('start: {}'.format(list_values[0]))
    print('end: {}'.format(list_values[-1]))
    payload = json.dumps({
        "value": list_values
    })
    return payload

def InsertBatchintoCogSearch(df):
    # Insert Batch into Azure AI Search
    url = f"https://{cog_search_name}.search.windows.net/indexes/{cog_search_index}/docs/index?api-version=2023-10-01-Preview"
    payload = batch_append_payload(df)
    headers = {
    'Content-Type': 'application/json',
    'api-key': cog_search_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.json())

    if response.status_code == 200 or response.status_code == 201:
        return "Success"
    else:
        return "Failure"
    
InsertBatchintoCogSearch(df)

payload of size 99
start: {'id': '1', 'contentVector': [-0.005931428167968988, -0.0029030460864305496, -0.016761111095547676, 0.0030588689260184765, -0.009152894839644432, 0.029782475903630257, -0.005121827125549316, -0.046475838869810104, -0.02475549653172493, -0.03260083496570587, -0.0016285174060612917, 0.04430786892771721, -0.02872559055685997, -0.007147523108869791, -0.026842167600989342, 0.007885987870395184, 0.03056836500763893, -0.0001247005711775273, -0.010440126061439514, -0.005585907958447933, -0.03165234997868538, 0.0008765031234361231, 0.00010098840721184388, -0.008983521722257137, -0.006598756182938814, -0.009810060262680054, 0.019403323531150818, -0.012655519880354404, -0.002560913562774658, 0.012133852578699589, 0.01616491936147213, 0.00803503580391407, -0.03569019213318825, -0.004637421574443579, 0.009396791458129883, 0.004962616600096226, -0.001298240851610899, -0.016828861087560654, 0.032655034214258194, -0.03135425224900246, -0.003338333684951067, 0.0227907747030258

'Success'

Lets convert a user question to an embedding and then retrieve the Top K chunks based on users question using the cosine similarity.

In [19]:
question = "I love those candies"
retrieve_topK_element = 4


In [20]:
# retrieve k chnuks
def retrieve_k_chunk(k, questionEmbedding):
    # Retrieve the top K entries
    url = f"https://{cog_search_name}.search.windows.net/indexes/{cog_search_index}/docs/search?api-version=2023-10-01-Preview"

    payload = json.dumps({
    "vectorQueries": [
        {
            "kind": "vector",
            "vector": questionEmbedding,
            "fields": "contentVector",
            "k": k
        }
    ]
    })
    headers = {
    'Content-Type': 'application/json',
    'api-key': cog_search_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    output = json.loads(response.text)
    print(response.status_code)
    return output

# Generate embeddings for the question and retrieve the top k document chunks
questionEmbedding = createEmbeddings(question)
output = retrieve_k_chunk(retrieve_topK_element, questionEmbedding)
print(len(output['value']))

200
4


In [21]:
# Use the top k ids to retrieve the actual text from the database 
top_ids = []
for i in range(len(output['value'])):
    top_ids.append(int(output['value'][i]['id']))

print(top_ids)

[24, 17, 23, 15]


In [22]:
# Assuming you have already established a connection and have a cursor object
top_ids_string = ', '.join(map(str, top_ids))

sql = f"SELECT CONCAT('productid: ', productid, ' ', 'score: ', score, ' ', 'text: ', text) AS concat FROM food_review WHERE Id IN ({top_ids_string})"

# Execute the SELECT statement
try:
    cursor.execute(sql)    
    top_rows = cursor.fetchall()
    for row in top_rows:
        print(row)
except (Exception) as e:
    print(f"Error executing SELECT statement: {e}")

('productid: B001GVISJM score: 5 text: The Strawberry Twizzlers are my guilty pleasure - yummy. Six pounds will be around for a while with my son and I.',)
('productid: B001GVISJM score: 2 text: I love eating them and they are good for watching TV and looking at movies! It is not too sweet. I like to transfer them to a zip lock baggie so they stay fresh so I can take my time eating them.',)
("productid: B001GVISJM score: 5 text: I can remember buying this candy as a kid and the quality hasn't dropped in all these years. Still a superb product you won't be disappointed with.",)
('productid: B001GVISJM score: 5 text: I love this candy.  After weight watchers I had to cut back but still have a craving for it.',)


In [23]:
# create a prompt template 
template = """
    context :{context}
    Answer the question based on the context above. Provide the product id associated with the answer as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Question: {query}
    Answer: """

In [24]:
# create the context from the top_rows
context = ""
for row in top_rows:
    context += row[0]
    context += "\n"
    
print(context)

productid: B001GVISJM score: 5 text: The Strawberry Twizzlers are my guilty pleasure - yummy. Six pounds will be around for a while with my son and I.
productid: B001GVISJM score: 2 text: I love eating them and they are good for watching TV and looking at movies! It is not too sweet. I like to transfer them to a zip lock baggie so they stay fresh so I can take my time eating them.
productid: B001GVISJM score: 5 text: I can remember buying this candy as a kid and the quality hasn't dropped in all these years. Still a superb product you won't be disappointed with.
productid: B001GVISJM score: 5 text: I love this candy.  After weight watchers I had to cut back but still have a craving for it.



In [25]:
print(question)
user_input = template.format(context=context, query=question)
print(user_input)



I love those candies

    context :productid: B001GVISJM score: 5 text: The Strawberry Twizzlers are my guilty pleasure - yummy. Six pounds will be around for a while with my son and I.
productid: B001GVISJM score: 2 text: I love eating them and they are good for watching TV and looking at movies! It is not too sweet. I like to transfer them to a zip lock baggie so they stay fresh so I can take my time eating them.
productid: B001GVISJM score: 5 text: I can remember buying this candy as a kid and the quality hasn't dropped in all these years. Still a superb product you won't be disappointed with.
productid: B001GVISJM score: 5 text: I love this candy.  After weight watchers I had to cut back but still have a craving for it.

    Answer the question based on the context above. Provide the product id associated with the answer as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Question: I love those candies
    Answer

In [26]:

response = chat_client.chat.completions.create(
    model="gpt-4", 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_input}
    ],
    max_tokens=1024,
    n=1,
    temperature=1
)
# response_message = response.choices[0].message
# print(response_message)
response_dict = response.model_dump()    # <--- convert to dictionary
response_message = response_dict["choices"][0]["message"]["content"]
print(response_message)

# git rm -r --cached .env                                                                                                               
# >> 
# >> git add .
# >> git commit -m 'your message'
# >> git push

Based on your statement, it seems you're referring to the product with id B001GVISJM.
